In [2]:
import os
import pandas as pd
import csv
import json
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image, ImageDraw
import random
import ast
from tqdm import tqdm

In [3]:
CSV_PATH = './tinyquickdraw_data/csv_trans/'
JSON_PATH = './tinyquickdraw_data/quickdraw_simplified'
DRAW_PATH = './tinyquickdraw_data/sketches/sketches/'
IMAGE_PATH = './tinyquickdraw_data/images/'

### ndjson to csv

In [ ]:
DATA_FLIE = os.listdir(JSON_PATH)

for cls in DATA_FLIE[:1]:
    print(cls)
    path = os.path.join(JSON_PATH, cls)
    csv_file_name = cls.split('.')[0] + '.csv'
    csv_save = CSV_PATH + csv_file_name
    print(f'{csv_file_name} processing...')

    with open(path, 'r', encoding='utf-8', newline="") as lines, open(csv_save, 'w', encoding='utf-8',newline="") as output_w:
        writer = csv.writer(output_w)
        writer.writerow(['key_id', 'drawing'])
        for line in lines:
            data = json.loads(line)
            if data['recognized'] == True:
                writer.writerow([data['key_id'], data['drawing']])

    print(f'{csv_file_name} Done...')

#### class 100개, 5000개씩 추출

In [11]:
import shutil

csv_save_dir = './tinyquickdraw_data/csvs'

csvs = os.listdir(CSV_PATH)
classes = [csv_file for csv_file in os.listdir(CSV_PATH)]
choice_cls = random.sample(classes, 100)
print(choice_cls)
for csv_f in choice_cls:
    shutil.copy2(os.path.join(CSV_PATH,csv_f), os.path.join(csv_save_dir, csv_f))


['golf club.csv', 'tent.csv', 'rhinoceros.csv', 'river.csv', 'pliers.csv', 'octagon.csv', 'mug.csv', 'cactus.csv', 'saw.csv', 'pig.csv', 'basket.csv', 'ear.csv', 'stove.csv', 'dragon.csv', 'pineapple.csv', 'peanut.csv', 'drill.csv', 'eraser.csv', 'house plant.csv', 'saxophone.csv', 'donut.csv', 'broccoli.csv', 'snowflake.csv', 'hamburger.csv', 'sea turtle.csv', 'line.csv', 'crown.csv', 'shovel.csv', 'ice cream.csv', 'mouse.csv', 'power outlet.csv', 'helmet.csv', 'camera.csv', 'knife.csv', 'door.csv', 'light bulb.csv', 'mushroom.csv', 'duck.csv', 'table.csv', 'broom.csv', 'van.csv', 'microwave.csv', 'see saw.csv', 'carrot.csv', 'church.csv', 'bicycle.csv', 'mermaid.csv', 'cloud.csv', 'sleeping bag.csv', 'ambulance.csv', 'fan.csv', 'rabbit.csv', 'pear.csv', 'couch.csv', 'paint can.csv', 'shark.csv', 'swing set.csv', 'blueberry.csv', 'swan.csv', 'potato.csv', 'piano.csv', 'The Great Wall of China.csv', 'kangaroo.csv', 'floor lamp.csv', 'barn.csv', 'cow.csv', 'truck.csv', 'traffic light.cs

#### key 별 csv 저장

In [13]:
save_key_data = './tinyquickdraw_data/key_data'
names = []
for csv_file in choice_cls:
    path = os.path.join(CSV_PATH, csv_file)
    csv_name = csv_file.split('.')[0]
    names.append(csv_name)

    rd_csv = pd.read_csv(path)
    pick_feature = rd_csv.sample(n=5000, replace=False)
    
    for i, data in tqdm(pick_feature.iterrows(), total=len(pick_feature)):
        pd.DataFrame([data['drawing']], columns=['drawing']).to_csv(f'{save_key_data}/{csv_name}_{data["key_id"]}.csv', index=False)
print(names)

100%|██████████| 5000/5000 [00:17<00:00, 277.84it/s]

['golf club', 'tent', 'rhinoceros', 'river', 'pliers', 'octagon', 'mug', 'cactus', 'saw', 'pig', 'basket', 'ear', 'stove', 'dragon', 'pineapple', 'peanut', 'drill', 'eraser', 'house plant', 'saxophone', 'donut', 'broccoli', 'snowflake', 'hamburger', 'sea turtle', 'line', 'crown', 'shovel', 'ice cream', 'mouse', 'power outlet', 'helmet', 'camera', 'knife', 'door', 'light bulb', 'mushroom', 'duck', 'table', 'broom', 'van', 'microwave', 'see saw', 'carrot', 'church', 'bicycle', 'mermaid', 'cloud', 'sleeping bag', 'ambulance', 'fan', 'rabbit', 'pear', 'couch', 'paint can', 'shark', 'swing set', 'blueberry', 'swan', 'potato', 'piano', 'The Great Wall of China', 'kangaroo', 'floor lamp', 'barn', 'cow', 'truck', 'traffic light', 'roller coaster', 'chandelier', 'angel', 'grapes', 'flip flops', 'speedboat', 'star', 'pillow', 'hedgehog', 'grass', 'zebra', 'asparagus', 'snowman', 'shoe', 'string bean', 'windmill', 'jacket', 'onion', 'submarine', 'fire hydrant', 'diving board', 'headphones', 'fenc

### 좌표 -> 이미지

In [ ]:
for cls in os.listdir(CSV_PATH)[309:310]:
    datas = os.path.join(CSV_PATH, cls)
    print(datas)
    data = pd.read_csv(datas)
    for coords in data.iloc[:, 1][2:3]:
        new_coord = []
        new = ast.literal_eval(coords)
        print(new)

        for coord in new:
            coord = np.array(coord)
            new_coord = coord.swapaxes(0, 1)
            print(new_coord)

In [ ]:
arrs = [[[184, 115, 67, 57, 36, 18], [251, 103, 12, 109, 193, 247]], [[145, 154, 150, 48, 24], [180, 176, 175, 178, 173]], [[55, 50, 38, 0, 28, 31, 60, 66, 71], [128, 152, 179, 253, 254, 247, 114, 26, 0]], [[72, 76, 81, 83, 86, 99, 93, 86], [26, 35, 71, 250, 255, 254, 189, 5]], [[67, 74, 97, 172], [12, 16, 49, 194]]]

def make_img(arrs): 
	image = Image.new("P", (256,256), color=255)
	image_d = ImageDraw.Draw(image)
	for stroke in arrs:
		for i in range(len(stroke[0]) - 1):
			image_d.line([stroke[0][i], 
						stroke[1][i], 
						stroke[0][i+1], 
						stroke[1][i+1]], 
						fill=0, width=5)
	return image

img = make_img(arrs) 
img = img.resize((64,64))
plt.imshow(img)

#### npz 이미지 열기 -> x

In [ ]:
for file in os.listdir(DRAW_PATH)[309:310]:
    path = os.path.join(DRAW_PATH, file)
    print(path)
    data = np.load(path, encoding='latin1', allow_pickle=True)
    print(data.files)
    
    print(data['test'].shape)
    print(data['train'].shape)
    print(data['valid'].shape)
    print(data['train'][0].shape)
    print(data['train'][1].shape)

    image = Image.fromarray(data['train'][0])   
    image.show()

#### npy 이미지 열기 -> x

In [ ]:
for image_file in os.listdir(IMAGE_PATH)[:1]:
    path = os.path.join(IMAGE_PATH, image_file)
    images = np.load(path, encoding="latin1", allow_pickle=True)
    print(images.shape)
    for image in images[:1]:
        img_arr = image.reshape(28, 28)
        img = Image.fromarray(img_arr)
        img.show()

#### 좌표로 이미지 그리기

In [ ]:
BASE_SIZE = 256

def draw_cv2(raw_strokes, size=256, lw=6, time_color=True):
    img = np.zeros((BASE_SIZE, BASE_SIZE), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        for i in range(len(stroke[0]) - 1):
            color = 255 - min(t, 10) * 13 if time_color else 255
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]),
                        (stroke[0][i + 1], stroke[1][i + 1]), color, lw)
    if size != BASE_SIZE:
        return cv2.resize(img, (size, size))
    else:
        return img

### 획마다 가중치
def draw_cv2_v2(raw_strokes, size=256, lw=6, augmentation = False):
    img = np.zeros((BASE_SIZE, BASE_SIZE, 3), np.uint8)
    for t, stroke in enumerate(raw_strokes):
        points_count = len(stroke[0]) - 1
        grad = 255//points_count
        for i in range(len(stroke[0]) - 1):
            _ = cv2.line(img, (stroke[0][i], stroke[1][i]), (stroke[0][i + 1], stroke[1][i + 1]), (255, 255 - min(t,10)*13, max(255 - grad*i, 20)), lw)
    if size != BASE_SIZE:
        img = cv2.resize(img, (size, size))
    if augmentation:
        if random.random() > 0.5:
            img = np.fliplr(img)
    return img


arrs = [[[184, 115, 67, 57, 36, 18], [251, 103, 12, 109, 193, 247]], [[145, 154, 150, 48, 24], [180, 176, 175, 178, 173]], [[55, 50, 38, 0, 28, 31, 60, 66, 71], [128, 152, 179, 253, 254, 247, 114, 26, 0]], [[72, 76, 81, 83, 86, 99, 93, 86], [26, 35, 71, 250, 255, 254, 189, 5]], [[67, 74, 97, 172], [12, 16, 49, 194]]]

# for arr in arrs:
x = draw_cv2(arrs[:1], size=224)
cv2.imshow('tt', x)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from keras.utils import pad_sequences
import ast

arrs = str(arrs)
stroke_vec = ast.literal_eval(arrs)
print(stroke_vec)
in_strokes = [(xi,yi,i) for i,(x,y) in enumerate(stroke_vec) for xi,yi in zip(x,y)]
print(in_strokes)
c_strokes = np.stack(in_strokes)
print(c_strokes[:5])
c_strokes[:,2] = [1]+np.diff(c_strokes[:,2]).tolist()
print("???", c_strokes)
c_strokes[:,2] += 1
print("+1", c_strokes)
c_strokes= c_strokes.swapaxes(0, 1)
print("swap", c_strokes)
seq = pad_sequences(c_strokes, maxlen=70, padding='post')
print("padding", seq)
seq = seq.swapaxes(0, 1)
print("swapping", seq)
print(seq.shape)

In [ ]:
import numpy as np
import torch
from torch.nn.utils.rnn import pack_padded_sequence

def pad_sequences(strokes, seq_lengths):
    seq_tensor = np.zeros((seq_lengths, 3))
    for idx, stroke in enumerate(strokes):
        seq_tensor[idx, :] = stroke
    # for idx, (seq, seqlen) in enumerate(zip(strokes, seq_lengths)):
    #     seq_tensor[idx, :seqlen] = torch.LongTensor(seq)
    return seq_tensor

arrs = [[[184, 115, 67, 57, 36, 18], [251, 103, 12, 109, 193, 247]], [[145, 154, 150, 48, 24], [180, 176, 175, 178, 173]], [[55, 50, 38, 0, 28, 31, 60, 66, 71], [128, 152, 179, 253, 254, 247, 114, 26, 0]], [[72, 76, 81, 83, 86, 99, 93, 86], [26, 35, 71, 250, 255, 254, 189, 5]], [[67, 74, 97, 172], [12, 16, 49, 194]]]

arrs = str(arrs)
stroke_vec = ast.literal_eval(arrs)
print(stroke_vec)
in_strokes = [(xi,yi,i) for i,(x,y) in enumerate(stroke_vec) for xi,yi in zip(x,y)]
print(in_strokes)
c_strokes = np.stack(in_strokes)
print(c_strokes[:5])
c_strokes[:,2] = [1]+np.diff(c_strokes[:,2]).tolist()
print("???", c_strokes)
c_strokes[:,2] += 1
print("+1", c_strokes)
# c_strokes= c_strokes.swapaxes(0, 1)
# print("swap", c_strokes)

print(pad_sequences(c_strokes, 70))